In [1]:
import networkx as nx
G = nx.Graph()

In [2]:
with open('hkg_network_post.txt', mode='r') as BabyFlightNodes:
    FlightEdges = BabyFlightNodes.read().split("%SECTION%")[2]
with open('hkg_network_post.txt', mode='r') as BabyFlightNodes:
    FlightNodes = BabyFlightNodes.read().split("%SECTION%")[1]

In [3]:
def GetTaxiway(FlightNodes, FlightEdges):
    '''
    Gets the Taxiway from Raw Data files showing the nodes and edges in Tuples
    '''
    FlightNodesCoords = list(FlightNodes.split("\n")[3:-1])
    FlightEdgesCoords = list(FlightEdges.split("\n")[3:])
    
    NodeID = []
    NodeXCoords = []
    NodeYCoords = []
    NodeLon = []
    NodeLat = []
    NodeName = []
    NodeType = []
    
    for item in FlightNodesCoords:
        if item == []:
            pass
        else:
            ItemSplit = list(item.split(";")[0:-1])
            try:  
                NodeID.append(ItemSplit[1])
                NodeXCoords.append(ItemSplit[2])
                NodeYCoords.append(ItemSplit[3])
                NodeLat.append(ItemSplit[4])
                NodeLon.append(ItemSplit[5])
                NodeName.append(ItemSplit[6])
                NodeType.append(ItemSplit[7])
            except IndexError:
                break
    FlightNodesList = list(zip(NodeID, NodeLat, NodeLon, NodeName, NodeType))
    
    EdgeID = [] 
    EdgeStart = [] 
    EdgeEnd = [] 
    EdgeDirected = [] 
    EdgeLength = [] 
    EdgeSpecification = [] 
    EdgeTraversalTime = [] 
    EdgeName = [] 
    
    for item in FlightEdgesCoords:
        
        if item == []:
            pass
        else:
            ItemSplit = list(item.split(";"))
            try:
                EdgeID.append(ItemSplit[1])
                EdgeStart.append(ItemSplit[2])
                EdgeEnd.append(ItemSplit[3])
                EdgeDirected.append(ItemSplit[4])
                EdgeLength.append(ItemSplit[5])
                EdgeSpecification.append(ItemSplit[6])
                EdgeTraversalTime.append(ItemSplit[7])
                EdgeName.append(ItemSplit[8])
            except IndexError:
                break
    FlightEdgesList = list(zip(EdgeID, EdgeStart, EdgeEnd, EdgeDirected, EdgeLength, EdgeSpecification, EdgeName))
    FlightEdgesWeights = list(zip(EdgeID, EdgeStart, EdgeEnd, EdgeLength))
    
    return (FlightNodesList, FlightEdgesList, FlightEdgesWeights)

In [4]:
FlightNodesList = GetTaxiway(FlightNodes, FlightEdges)[0]
FlightEdgesList = GetTaxiway(FlightNodes, FlightEdges)[1]
FlightEdgesWeights = GetTaxiway(FlightNodes, FlightEdges)[2]

In [5]:
#This will unpack each tuple to call any information about the list of Edges and Nodes
def GetEdgeID(FlightEdgesList):
    return [x[0] for x in FlightEdgesList]

def GetEdgeStart(FlightEdgesList, FlightNodesList):
    FlightEdgesStartNodes = [x[1] for x in FlightEdgesList]
    index = 0
    EdgeStartPoint = []
    for item in [x[1] for x in FlightEdgesList]:
        counter = 0
        for value in [x[0] for x in FlightNodesList]:
        
            if item == value:
                EdgeStartPoint.append((FlightNodesList[counter][1], FlightNodesList[counter][2]))
            counter += 1
        index += 1
    return (EdgeStartPoint)

def GetEdgeEnd(FlightEdgesList, FlightNodesList):
    FlightEdgesEndNodes = [x[2] for x in FlightEdgesList]
    index = 0
    EdgeEndPoint = []
    for item in [x[2] for x in FlightEdgesList]:
        counter = 0
        for value in [x[0] for x in FlightNodesList]:
        
            if item == value:
                EdgeEndPoint.append((FlightNodesList[counter][1], FlightNodesList[counter][2]))
            counter += 1
        index += 1
    return (EdgeEndPoint)

def GetEdgeLength(FlightEdgesList):
    return [x[4] for x in FlightEdgesList]

def GetEdgeSpecification(FlightEdgesList):
    return [x[5] for x in FlightEdgesList]


In [6]:
[x[1] for x in FlightEdgesList];

In [7]:
#Now that we have our data in Tuples, we can start making the Node Map with dictionaries

def GetNodeLinks(FlightNodes, FlightEdges):
    '''
    Returns Nodes with edges connected on a Graph in a dictionary
    '''
    Graph = {}
    for item in FlightNodes:
        Graph[item[0]] = []
        
    for item in FlightEdges:
        
        if item[1] in Graph.keys():
            Graph[item[1]].append(item[2])
        if item[2] in Graph.keys():
            Graph[item[2]].append(item[1])
    
    return (Graph)

In [8]:
BabyFlightLinks = GetNodeLinks(FlightNodesList, FlightEdgesList)
BabyFlightLinks;


In [10]:
def GetEdgesAndNodes(FlightEdgesList, FlightNodesList):
    '''
    Returns a list of tuples which give all info on edges including the coordinates of start and end.
    '''
    
    EdgeIDList = GetEdgeID(FlightEdgesList)
    EdgeStartList = GetEdgeStart(FlightEdgesList, FlightNodesList)
    EdgeEndList = GetEdgeEnd(FlightEdgesList, FlightNodesList)
    EdgeLengthList = GetEdgeLength(FlightEdgesList)
    EdgeSpecificationList = GetEdgeSpecification(FlightEdgesList)
    EdgeAndNodes = []
    for index, edge in enumerate(FlightEdgesList):
        try:
            EdgeID = int(EdgeIDList[index])
            NodeStart = int(edge[1])
            NodeEnd = int(edge[2])
            EdgeStart = EdgeStartList[index]
            EdgeEnd = EdgeEndList[index]
            EdgeLength = float(EdgeLengthList[index])
            EdgeSpecification = EdgeSpecificationList[index]
        except IndexError:
            break
        
        StartX = float(EdgeStart[0])
        StartY = float(EdgeStart[1])
        EndX = float(EdgeEnd[0])
        EndY = float(EdgeEnd[1])
        #This part below is important for calling things
        EdgeAndNodes.append((EdgeID, NodeStart, NodeEnd, EdgeLength, EdgeSpecification, StartX, StartY, EndX, EndY))
    return (EdgeAndNodes)